In [ ]:
### Import Libraries.

library("SingleCellExperiment")
library("slingshot")
library("Seurat")
library("RColorBrewer")
library("tidyverse")
library("writexl")

In [ ]:
### Load Data.

setwd("/folder/")
Seurat <- readRDS("Seurat.rds")

In [ ]:
### Seurat to SingleCellExperiment.

counts_matrix <- GetAssayData(Seurat, layer = "counts")
coldata <- Seurat@meta.data
sce <- SingleCellExperiment(assays = list(counts = counts_matrix), colData = coldata)

In [ ]:
### Add Embeddings.

umap <- Seurat@reductions$umap@cell.embeddings
reducedDims(sce)$UMAP <- umap

In [ ]:
### Run Slingshot.

sce <- slingshot(
  sce,
  clusterLabels = "Cluster_Column",
  reducedDim = "UMAP",
  start.clus = "RG" ### Or Olig_1
)

In [ ]:
### Plot Pseudotime Along Trajectory.

colors <- colorRampPalette(brewer.pal(11, 'Spectral')[-6])(100)
plotcol <- colors[cut(sce$slingPseudotime_1, breaks = 100)]

plot(reducedDims(sce)$UMAP, col = plotcol, pch = 16, asp = 1)
lines(SlingshotDataSet(sce), lwd = 2, col = 'black')

In [ ]:
### Plot Pseudotime Distribution Across Clusters.

df <- as.data.frame(colData(sce))
pseudotime_means <- df %>%
  group_by(Cluster_Column) %>%
  summarise(mean_pseudotime = mean(slingPseudotime_1, na.rm = TRUE))